# The Biventricular (BiV) Model

written by _Avan Suinesiaputra - 2025_

----

A biventricular model consists of left (LV) and right (RV) ventricles of the heart. 
The `biv` model defined in this `biv-lite` library can only be used from a _fitted heart model_, which
contains **388 control points** of $x$, $y$, and $z$ coordinates.

An example of a fitted model is given in the `tests/fitted_model.txt` file.

In [2]:
import numpy as np
import rich

In [3]:
model_file = '../tests/fitted_model.txt'

control_points = np.loadtxt(model_file, delimiter=',',skiprows=1, usecols=[0,1,2]).astype(float)
print(control_points)

[[  85.77214372  -62.79107797  -68.03511961]
 [  -5.28144593  -23.28242565  -18.21744542]
 [  -2.71527901  -19.47093943  -18.2818516 ]
 ...
 [ -29.55247199  -49.02128846  -41.37874376]
 [ -23.92840493  -50.7869584   -50.88843623]
 [  52.21450523 -109.90147822  -33.57802701]]


In [4]:
print(control_points.shape)

(388, 3)


> There are 388 rows in the `control_points` matrix with columns are $x$, $y$, and $z$ coordinate points.

Plotting the control points directly won't give you any heart shape visual.

In [5]:
import pyvista as pv
pv.set_jupyter_backend('html')
pv.start_xvfb()

In [6]:
pv.PolyData(control_points).plot(point_size=5, style="points", color="crimson")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Creating a `BivMesh` object

The `BivMesh` class requires an input of $388\times 3$ control point matrix.

In [7]:
import sys
sys.path.append('../src/')

from biv_mesh import BivMesh, Components

In [8]:
biv = BivMesh(control_points, name="my_first_biv_model")
print(biv)

In [9]:
rich.print({
    'name': biv.label,
    'number_of_nodes': biv.nb_nodes,
    'node_basis': biv.nodes_basis,
    'nodes': [biv.nodes.shape, biv.nodes.dtype],
    'number_of_elements': biv.nb_elements,
    'elements': [biv.elements.shape, biv.elements.dtype],
    'materials': [biv.materials.shape, biv.materials.dtype]
})

{
    'name': 'my_first_biv_model',
    'number_of_nodes': 5810,
    'node_basis': 4,
    'nodes': [(5810, 3), dtype('float64')],
    'number_of_elements': 11920,
    'elements': [(11920, 3), dtype('int64')],
    'materials': [(11920,), dtype('float64')]
}

The `BivMesh` class implements the _subdivision process_, where elements defined specifically by the control points are subdivided into a smoother surface. The results are 11,920 elements with 5,810 vertices. Please read Charlène Mauger PhD thesis in chapter 3 for further detail of the subdivision surface method.

The `BivMesh` class inherits the generic `Mesh` class. Have a look into both class definition file to know more functionalities provided as methods and attributes.

## Plotting a `BivMesh` object

We can plot directly as a mesh, but the _pyvista_ library used in this notebook requires to put the number of elements in each row of the face matrix.

Let's create a generic function to create a pyvista face matrix:

In [10]:
def to_pyvista_faces(elements: np.ndarray) -> np.ndarray:
    return np.hstack([np.ones((elements.shape[0], 1)) * 3, elements]).astype(np.int32)

Then we can plot a biventricular mesh object

In [11]:
pl = pv.Plotter()
pl.add_mesh(pv.PolyData(biv.nodes, to_pyvista_faces(biv.elements)))
pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Advanced plotting

The model contains anatomical mesh components that are defined by the `Component` type:

There are several methods in the `BivMesh` class that you can use to retrieve these components. For example, to plot them in different colors:

In [12]:
pl = pv.Plotter()

# get the LV endocardial mesh
lv = biv.lv_endo()
pl.add_mesh(pv.PolyData(lv.nodes, to_pyvista_faces(lv.elements)), color="firebrick")

# get the RV endocardial mesh
rv = biv.rv_endo()
pl.add_mesh(pv.PolyData(rv.nodes, to_pyvista_faces(rv.elements)), color="dodgerblue")

# get the outer epicardial mesh
epi = biv.rvlv_epi()
pl.add_mesh(pv.PolyData(epi.nodes, to_pyvista_faces(epi.elements)), color="forestgreen", opacity=0.6, 
            show_edges=True, edge_color='black')

pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

You can directy get the meshes using `get_mesh_component` function:

In [13]:
pl = pv.Plotter()

# LV endocardial surface with firebrick color
lv_endo = biv.get_mesh_component([Components.LV_ENDOCARDIAL], label="LV_ENDO")
pl.add_mesh(pv.PolyData(lv_endo.nodes, to_pyvista_faces(lv_endo.elements)), color="firebrick", opacity=0.7)

# RV freewall surface with dodgerblue color
rv_fwall = biv.get_mesh_component([Components.RV_FREEWALL], label="RV_FREEWALL")
pl.add_mesh(pv.PolyData(rv_fwall.nodes, to_pyvista_faces(rv_fwall.elements)), color="dodgerblue", opacity=0.7)

# Septum with golden rod color
septum = biv.get_mesh_component([Components.RV_SEPTUM], label="SEPTUM")
pl.add_mesh(pv.PolyData(septum.nodes, to_pyvista_faces(septum.elements)), color="goldenrod", show_edges=True)

pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…